In [1]:
from pyspark.sql import SparkSession, functions as F
import os
from pyspark.sql.functions import explode
import pandas as pd
import pickle

In [2]:
spark = SparkSession.builder \
    .appName("reco") \
    .config("spark.master", "local[*]") \
    .getOrCreate()

In [3]:
# Function to get all the files in a directory
def get_files(path):
    print("Getting files from {}".format(path))
    files = []
    for file in os.listdir(path):
        if file.endswith(".json"):
            files.append(os.path.join(path, file))

    # Group the files in batches of 10
    batches = []
    for i in range(0, len(files), 5):
        batches.append(files[i : i + 5])
        print("Batch {} - {}".format(i, i + 5), " ready to be processed.")
        print("------------------")

    return batches

In [4]:
# Read the data
path = "Dataset/spotify_playlists"
groups = get_files(path)

Getting files from Dataset/spotify_playlists


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Dataset/spotify_playlists'

In [ ]:
# Group the files in batches of 10 to avoid memory issues
for group in groups[:10]:
    print("------------------")

    print("Reading group: ", groups.index(group) + 1, " of ", len(groups))
    data = spark.read.json(group, multiLine=True)
    data.createOrReplaceTempView("data")
    print("Finished reading group: ", groups.index(group) + 1, " of ", len(groups))

    playlist = data.select(explode("playlists.tracks").alias("tracks"))
    df = playlist.select("tracks.track_name", "tracks.track_uri", "tracks.artist_uri", "tracks.album_uri")

    # Add a column with the index of the playlist
    df = df.withColumn("playlist_id", F.monotonically_increasing_id())

    # Cache the dataframe in memory
    df.cache()

    # Define a function to convert a row of the dataframe to a list of songs
    def row_to_songs(row):
        # Go through each value in the row and append it to the list
        songs_df = []
        for i in range(len(row['track_name'])):
            songs_df.append([row['track_name'][i], row['track_uri'][i], row['artist_uri'][i], row['album_uri'][i], row['playlist_id']])
        return songs_df

    print("------------------")
    
    # Use df.rdd.map() to apply the row_to_songs() function to each row of the dataframe in parallel
    all_songs_rdd = df.rdd.map(row_to_songs)

    # Convert the RDD to a list
    all_songs = all_songs_rdd.collect()

    # print("Finish converting the dataframe to a list of songs in group: ", str(groups.index(group)))

    # Save the file to disk
    with open('Dataset/temp_data/' + str(groups.index(group)) + '.pkl', 'wb') as f:
        pickle.dump(all_songs, f)
        print("Document saved to disk")


    # Unpersist the dataframe from memory
    df.unpersist()

    print("------------------")


In [11]:
# Group the files in batches of 10 to avoid memory issues, and save the data to disk
for group in groups[10:]:
    print("------------------")

    print("Reading group: ", groups.index(group) + 1, " of ", len(groups))
    data = spark.read.json(group, multiLine=True)
    data.createOrReplaceTempView("data")
    print("Finished reading group: ", groups.index(group) + 1, " of ", len(groups))

    playlist = data.select(explode("playlists.tracks").alias("tracks"))
    df = playlist.select("tracks.track_name", "tracks.track_uri", "tracks.artist_uri", "tracks.album_uri")

    # Add a column with the index of the playlist
    df = df.withColumn("playlist_id", F.monotonically_increasing_id())

    # Cache the dataframe in memory
    df.cache()

    # Define a function to convert a row of the dataframe to a list of songs
    def row_to_songs(row):
        # Go through each value in the row and append it to the list
        songs_df = []
        for i in range(len(row['track_name'])):
            songs_df.append([row['track_name'][i], row['track_uri'][i], row['artist_uri'][i], row['album_uri'][i], row['playlist_id']])
        return songs_df

    print("------------------")
    
    # Use df.rdd.map() to apply the row_to_songs() function to each row of the dataframe in parallel
    all_songs_rdd = df.rdd.map(row_to_songs)

    # Convert the RDD to a list
    all_songs = all_songs_rdd.collect()

    # print("Finish converting the dataframe to a list of songs in group: ", str(groups.index(group)))

    # Save the file to disk
    with open('Dataset/temp_data/' + str(groups.index(group)) + '.pkl', 'wb') as f:
        pickle.dump(all_songs, f)
        print("Document saved to disk")


    # Unpersist the dataframe from memory
    df.unpersist()

    print("------------------")


Getting files from Dataset/spotify_playlists
Batch 0 - 5  ready to be processed.
------------------
Batch 5 - 10  ready to be processed.
------------------
Batch 10 - 15  ready to be processed.
------------------
Batch 15 - 20  ready to be processed.
------------------
Batch 20 - 25  ready to be processed.
------------------
Batch 25 - 30  ready to be processed.
------------------
Batch 30 - 35  ready to be processed.
------------------
Batch 35 - 40  ready to be processed.
------------------
Batch 40 - 45  ready to be processed.
------------------
Batch 45 - 50  ready to be processed.
------------------
Batch 50 - 55  ready to be processed.
------------------
Batch 55 - 60  ready to be processed.
------------------
Batch 60 - 65  ready to be processed.
------------------
Batch 65 - 70  ready to be processed.
------------------
Batch 70 - 75  ready to be processed.
------------------
Batch 75 - 80  ready to be processed.
------------------
Batch 80 - 85  ready to be processed.
--------

Finished reading group:  11  of  20
------------------
Exploding the data
Finished exploding the data
------------------
------------------
Converting the dataframe to a list of songs


Document saved to disk
------------------
------------------
------------------
------------------
Reading group:  12  of  20


Finished reading group:  12  of  20
------------------
Exploding the data
Finished exploding the data
------------------
------------------
Converting the dataframe to a list of songs


Document saved to disk
------------------
------------------
------------------
------------------
Reading group:  13  of  20


...

Document saved to disk
------------------
------------------
------------------
------------------
Reading group:  19  of  20


Finished reading group:  19  of  20
------------------
Exploding the data
Finished exploding the data
------------------
------------------
Converting the dataframe to a list of songs


Document saved to disk
------------------
------------------
------------------
------------------
Reading group:  20  of  20


Finished reading group:  20  of  20
------------------
Exploding the data
Finished exploding the data
------------------
------------------
Converting the dataframe to a list of songs


Document saved to disk
------------------
------------------


In [ ]:
# Ge the column names of the all_songs list
columns = ['track_name', 'track_uri', 'artist_uri', 'album_uri', 'playlist_id']

In [ ]:
# Merge all the files pkl files into one list
all_songs_merge = []
for group in groups[:10]:
    print("------------------")
    with open('Dataset/temp_data/' + str(groups.index(group)) + '.pkl', 'rb') as f:
        all_songs_merge.extend(pickle.load(f)) 
        print("Document loaded from disk", "Group: ", str(groups.index(group)))

        # When read the file, delete the file
        # os.remove('Dataset/temp_data/' + str(groups.index(group)) + '.pkl')
        # print("Document deleted from disk")
    print("------------------")

print("Finished merging all the files")

print("Converting the list to a dataframe")
# Convert the list to a dataframe
all_songs_df = pd.DataFrame([item for sublist in all_songs_merge for item in sublist], columns=columns)

print("Data frame created, shape: ", all_songs_df.shape)

# Merge the duplicate rows into one adding the playlists to the playlists column in a array
all_songs_df_grouped = all_songs_df.groupby(['track_name', 'track_uri', 'artist_uri', 'album_uri'])['playlist_id'].apply(list).reset_index()

print("Data grouped, shape: ", all_songs_df_grouped.shape)


# new column with the number of playlists
all_songs_df_grouped['playlist_count'] = all_songs_df_grouped['playlist_id'].apply(lambda x: len(set(x)))

# Sort the dataframe by the number of playlistsç
all_songs_df_grouped = all_songs_df_grouped.sort_values(by=['playlist_count'], ascending=False)

# Safe the dataframe to a csv file with the name of the group
all_songs_df_grouped.to_csv('Dataset/datafull1.csv', index=False)

print("Data saved to csv")
print("------------------")
print("Finished processing the data")

In [12]:
# Merge all the files pkl files into one list
all_songs_merge = []
for group in groups[10:]:
    print("------------------")
    with open('Dataset/temp_data/' + str(groups.index(group)) + '.pkl', 'rb') as f:
        all_songs_merge.extend(pickle.load(f)) 
        print("Document loaded from disk", "Group: ", str(groups.index(group)))

        # When read the file, delete the file
        # os.remove('Dataset/temp_data/' + str(groups.index(group)) + '.pkl')
        # print("Document deleted from disk")
    print("------------------")

print("Finished merging all the files")

print("Converting the list to a dataframe")
# Convert the list to a dataframe
all_songs_df = pd.DataFrame([item for sublist in all_songs_merge for item in sublist], columns=columns)

print("Data frame created, shape: ", all_songs_df.shape)

# Merge the duplicate rows into one adding the playlists to the playlists column in a array
all_songs_df_grouped = all_songs_df.groupby(['track_name', 'track_uri', 'artist_uri', 'album_uri'])['playlist_id'].apply(list).reset_index()

print("Data grouped, shape: ", all_songs_df_grouped.shape)


# new column with the number of playlists
all_songs_df_grouped['playlist_count'] = all_songs_df_grouped['playlist_id'].apply(lambda x: len(set(x)))

# Sort the dataframe by the number of playlistsç
all_songs_df_grouped = all_songs_df_grouped.sort_values(by=['playlist_count'], ascending=False)

# Safe the dataframe to a csv file with the name of the group
all_songs_df_grouped.to_csv('Dataset/datafull2.csv', index=False)

print("Data saved to csv")
print("------------------")
print("Finished processing the data")

------------------
Document loaded from disk Group:  10
------------------
------------------
Document loaded from disk Group:  11
------------------
------------------
Document loaded from disk Group:  12
------------------
------------------
Document loaded from disk Group:  13
------------------
------------------
Document loaded from disk Group:  14
------------------
------------------
Document loaded from disk Group:  15
------------------
------------------
Document loaded from disk Group:  16
------------------
------------------
Document loaded from disk Group:  17
------------------
------------------
Document loaded from disk Group:  18
------------------
------------------
Document loaded from disk Group:  19
------------------
Finished merging all the files
Converting the list to a dataframe
Data frame created, shape:  (3331480, 5)
Data grouped, shape:  (456346, 5)
Data saved to csv
------------------
Finished processing the data


In [1]:
# Merge datafull2.csv with datafull1.csv, they have in common the columns: track_name, track_uri, artist_uri, album_uri, then combine the playlists_id and playlist_count columns
import pandas as pd
# Read the data
df1 = pd.read_csv('Dataset/datafull1.csv')
df2 = pd.read_csv('Dataset/datafull2.csv')

In [2]:
import ast
df1["playlist_id"] = df1["playlist_id"].apply(lambda x: ast.literal_eval(x))
df2["playlist_id"] = df2["playlist_id"].apply(lambda x: ast.literal_eval(x))

# change the type of the values on each array in the arrays in the playlist_id column
df1["playlist_id"] = df1["playlist_id"].apply(lambda x: [str(i) for i in x])
df2["playlist_id"] = df2["playlist_id"].apply(lambda x: [str(i) for i in x])

In [3]:
import pandas as pd
# print the number of rows in each dataframe
print("Number of rows in df1: ", df1.shape[0])
print("Number of rows in df2: ", df2.shape[0])
# total number of rows
print("Total number of rows: ", df1.shape[0] + df2.shape[0])
# How many songs are in common
print("Number of songs in common: ", len(set(df1['track_uri']).intersection(set(df2['track_uri']))))

# Outer join length
print("Outer join length: ", len(set(df1['track_uri']).union(set(df2['track_uri']))))

# outer join + common songs
print("Outer join + common songs: ", len(set(df1['track_uri']).union(set(df2['track_uri']))) + len(set(df1['track_uri']).intersection(set(df2['track_uri']))))

Number of rows in df1:  462812
Number of rows in df2:  456346
Total number of rows:  919158
Number of songs in common:  237353
Outer join length:  681805
Outer join + common songs:  919158


In [4]:
df_common = pd.merge(df1, df2, how='inner', on=['track_name', 'track_uri', 'artist_uri', 'album_uri'])
# Concatenate the playlist_id columns
df_common['playlist_id'] = df_common['playlist_id_x'] + df_common['playlist_id_y']
# compute the number of playlists for x and y
df_common['playlist_count_x'] = df_common['playlist_id_x'].apply(lambda x: len(set(x)))
df_common['playlist_count_y'] = df_common['playlist_id_y'].apply(lambda x: len(set(x)))
# new column with the combination of playlist_id_x and playlist_id_y unique values in a column id  and the number of playlists in a column count
df_common['playlist_id'] = df_common['playlist_id'].apply(lambda x: list(set(x)))
df_common['playlist_count'] = df_common['playlist_id'].apply(lambda x: len(set(x)))

# Drop the columns playlist_id_x, playlist_id_y, playlist_count_x, playlist_count_y
df_common = df_common.drop(columns=['playlist_id_x', 'playlist_id_y', 'playlist_count_x', 'playlist_count_y'])

In [202]:
len(df_common)

237353

In [5]:
# Outter minus join
df_outter = pd.merge(df1, df2, how='outer', on=['track_name', 'track_uri', 'artist_uri', 'album_uri'], indicator=True)
# Select the rows that are only in df1
df_outter_only_df1 = df_outter[df_outter['_merge'] == 'left_only']
# Select the rows that are only in df2
df_outter_only_df2 = df_outter[df_outter['_merge'] == 'right_only']

# Concatenate the playlist_id columns
df_outter_only_df1['playlist_id'] = df_outter_only_df1['playlist_id_x']
df_outter_only_df2['playlist_id'] = df_outter_only_df2['playlist_id_y']

# compute the number of playlists for x and y
df_outter_only_df1['playlist_count'] = df_outter_only_df1['playlist_id_x'].apply(lambda x: len(set(x)))
df_outter_only_df2['playlist_count'] = df_outter_only_df2['playlist_id_y'].apply(lambda x: len(set(x)))

# Drop the columns that are not needed
df_outter_only_df1 = df_outter_only_df1.drop(['playlist_id_x', 'playlist_id_y', 'playlist_count_x', 'playlist_count_y', '_merge'], axis=1)
df_outter_only_df2 = df_outter_only_df2.drop(['playlist_id_x', 'playlist_id_y', 'playlist_count_x', 'playlist_count_y', '_merge'], axis=1)

# Concatenate the two dataframes
df_outter = pd.concat([df_outter_only_df1, df_outter_only_df2])


C:\Users\juan\AppData\Local\Temp\ipykernel_9640\4069587794.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outter_only_df1['playlist_id'] = df_outter_only_df1['playlist_id_x']
C:\Users\juan\AppData\Local\Temp\ipykernel_9640\4069587794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outter_only_df2['playlist_id'] = df_outter_only_df2['playlist_id_y']
C:\Users\juan\AppData\Local\Temp\ipykernel_9640\4069587794.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [203]:
len(df_outter)

444452

In [7]:
df_common[df_common.duplicated(subset=['track_uri'], keep=False)]


,track_name,track_uri,artist_uri,album_uri,playlist_id,playlist_count


In [6]:
# find duplicates in df_outter
df_outter[df_outter.duplicated(subset=['track_uri'], keep=False)]




,track_name,track_uri,artist_uri,album_uri,playlist_id,playlist_count


In [112]:
# Concatenate the two dataframes
df_final = pd.concat([df_common, df_outter])
df_final

,track_name,track_uri,artist_uri,album_uri,playlist_id,playlist_count
0,HUMBLE.,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg,spotify:album:4eLPsYPBmXABThSJ821sqY,"[17179869726, 34359739337, 17179869700, 171798...",2929
1,Closer,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,spotify:artist:69GGBxA162lTqCwzJG5jLp,spotify:album:0rSLgV8p5FzfnqlEk4GzxE,"[190, 34359738397, 17179869700, 17179869260, 3...",2832
2,One Dance,spotify:track:1xznGGDReH1oQq0xzbwXa3,spotify:artist:3TVXtAsR1Inumwj472S9r4,spotify:album:3hARKC8cinq3mZLLAEaBh9,"[17179869726, 190, 17179869260, 8589935195, 34...",2825
3,Broccoli (feat. Lil Yachty),spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,spotify:artist:5M0lbkGluOPXLeFjApw8r8,spotify:album:0NrZHZ0y5kTO8EHliuUUca,"[34359738917, 190, 34359738448, 8589935195, 34...",2793
4,Congratulations,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,spotify:artist:246dkjvS1zLTtiykXe5h60,spotify:album:5s0rmjP8XOPhP6HhqOhuyC,"[34359738917, 190, 17179869700, 34359738397, 3...",2733
...,...,...,...,...,...,...
681800,Intro Bonito,spotify:track:19pUiFzM1Lfui5IzvbJInJ,spotify:artist:6OqhFYFJDnBBHas02HopPT,spotify:album:2xyl7MxdxOGd7m8qV6mtcM,[8589935151],1
681801,Intro / Mule Skinner Blues (Blue Yodel No. 8) ...,spotify:track:3IOJXkHcwFDTOSLxcAw2XF,spotify:artist:64vAECmFoB6mi7n1zTRwR8,spotify:album:1XXMBnpx20DkGNyiuf4TPa,[25769804664],1
681802,Intro / Madness,spotify:track:62KdOlGDRX7jbQJLIivG5j,spotify:artist:75S63f1AmZUa9gpQvlt5NB,spotify:album:6etmKrHivh8W7SouljN2si,[8589934677],1
681803,Intro - Urban Salute to Hector Lavoe,spotify:track:5krfvR6iPmNbXalFgmtvPS,spotify:artist:6P9Adm5Ne2YtzhV1hOjQcC,spotify:album:4Hf8SuY3DNK07Q8cagjCNR,[17179869244],1


In [76]:
df_final[df_final.duplicated(subset=['track_uri'], keep=False)]

,track_name,track_uri,artist_uri,album_uri,playlist_id,playlist_count


In [113]:
# Drop the columns that are not needed  track_name, artist_uri, album_uri,
df_final = df_final.drop('album_uri', axis=1)

In [114]:
# Modify the column Track_uri, remove the spotify:track: from the values
df_final['track_uri'] = df_final['track_uri'].apply(lambda x: x.replace('spotify:track:', ''))
# Modify the column artist_uri, remove the spotify:artist: from the values
df_final['artist_uri'] = df_final['artist_uri'].apply(lambda x: x.replace('spotify:artist:', ''))

In [80]:
# Read tje track data
df_tracks = pd.read_csv('Dataset/tracks.csv')

In [115]:
# Rename the id column to track_uri and name to track_name
df_tracks = df_tracks.rename(columns={'id': 'track_uri', 'name': 'track_name', 'id_artists': 'artist_uri'})

# Modify the column Track_uri, remove the []

In [122]:
# Merge the two dataframes on the track_uri column, if there is a match, the values will be added to the dataframe
# If there is no match, we will keep both dataframes and add NaN values
df_final_metadata = pd.merge(df_final, df_tracks, how='outer', on=['track_uri'])

In [85]:
df_final_metadata

,track_name_x,track_uri,playlist_id,playlist_count,track_name_y,popularity,duration_ms,explicit,artists,id_artists,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,HUMBLE.,7KXjTSCq5nL1LoYtL7XAwS,"[17179869726, 34359739337, 17179869700, 171798...",2929.0,HUMBLE.,83.0,177000.0,1.0,['Kendrick Lamar'],['2YZyLoL8N0Wb9xBt1NhZWg'],...,1.0,-6.638,0.0,0.1020,0.000282,0.000054,0.0958,0.4210,150.011,4.0
1,Closer,7BKLCZ1jbUBVqRi2FVlTVw,"[190, 34359738397, 17179869700, 17179869260, 3...",2832.0,Closer,86.0,244960.0,0.0,"['The Chainsmokers', 'Halsey']","['69GGBxA162lTqCwzJG5jLp', '26VFTg2z8YR0cCuwLz...",...,8.0,-5.599,1.0,0.0338,0.414000,0.000000,0.1110,0.6610,95.010,4.0
2,One Dance,1xznGGDReH1oQq0xzbwXa3,"[17179869726, 190, 17179869260, 8589935195, 34...",2825.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Broccoli (feat. Lil Yachty),7yyRTcZmCiyzzJlNzGC9Ol,"[34359738917, 190, 34359738448, 8589935195, 34...",2793.0,Broccoli (feat. Lil Yachty),68.0,225205.0,1.0,"['Shelley FKA DRAM', 'Lil Yachty']","['5M0lbkGluOPXLeFjApw8r8', '6icQOAFXDZKsumw3YX...",...,8.0,-7.390,1.0,0.1310,0.236000,0.000000,0.0570,0.7080,145.990,4.0
4,Congratulations,3a1lNhkSLSkpJE4MSHpDu9,"[34359738917, 190, 17179869700, 34359738397, 3...",2733.0,Congratulations,83.0,220293.0,1.0,"['Post Malone', 'Quavo']","['246dkjvS1zLTtiykXe5h60', '0VRj0yCOv2FXJNP47X...",...,6.0,-4.183,1.0,0.0363,0.215000,0.000000,0.2530,0.4920,123.146,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207057,NaN,5rgu12WBIHQtvej2MdHSH0,NaN,NaN,云与海,50.0,258267.0,0.0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],...,0.0,-7.471,0.0,0.0292,0.785000,0.000000,0.0648,0.2110,131.896,4.0
1207058,NaN,0NuWgxEp51CutD2pJoF4OM,NaN,NaN,blind,72.0,153293.0,0.0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],...,0.0,-5.223,1.0,0.0652,0.141000,0.000297,0.0924,0.6860,150.091,4.0
1207059,NaN,27Y1N4Q4U3EfDU5Ubw8ws2,NaN,NaN,What They'll Say About Us,70.0,187601.0,0.0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],...,7.0,-12.823,0.0,0.0408,0.895000,0.000150,0.0874,0.0663,145.095,4.0
1207060,NaN,45XJsGpFTyzbzeWK8VzR8S,NaN,NaN,A Day At A Time,58.0,142003.0,0.0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",...,10.0,-6.212,1.0,0.0345,0.206000,0.000003,0.3050,0.4380,90.029,4.0


In [117]:
df_final_metadata.columns

Index(['track_name_x', 'track_uri', 'artist_uri_x', 'playlist_id',
       'playlist_count', 'track_name_y', 'popularity', 'duration_ms',
       'explicit', 'artists', 'artist_uri_y', 'release_date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [87]:
# Unique tracks_uri indf_final_metadata
len(df_final_metadata['track_uri'].unique())

1207062

In [123]:
# Rename the column track_name_x to track_name if there is a match, if nan values, the column will be renamed to track_name_y
df_final_metadata = df_final_metadata.rename(columns={'track_name_x': 'track_name'})

# if track_name is nan, the value of track_name_y will be assigned to track_name
df_final_metadata['track_name'] = df_final_metadata.apply(lambda x: x['track_name_y'] if pd.isnull(x['track_name']) else x['track_name'], axis=1)

# Rwmove the column artist_uri_x to artist_uri if there is a match, if nan values, the column will be renamed to artist_uri_y
df_final_metadata = df_final_metadata.rename(columns={'artist_uri_y': 'artist_uri'})

# if artist_uri is nan, the value of artist_uri_y will be assigned to artist_uri
df_final_metadata['artist_uri'] = df_final_metadata.apply(lambda x: x['artist_uri_x'] if pd.isnull(x['artist_uri']) else x['artist_uri'], axis=1)


# Drop the columns that are not needed
df_final_metadata = df_final_metadata.drop(['track_name_y', 'artist_uri_x'], axis=1)

In [128]:
len(df_final_metadata)

1207062

In [139]:
# Get the number of tracks that have null values in the column playlist_id
df_final_metadata[df_final_metadata['explicit'].isnull()]

,track_name,track_uri,playlist_id,playlist_count,popularity,duration_ms,explicit,artists,artist_uri,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2,One Dance,1xznGGDReH1oQq0xzbwXa3,"[17179869726, 190, 17179869260, 8589935195, 34...",2825.0,NaN,NaN,NaN,NaN,3TVXtAsR1Inumwj472S9r4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Roses,6O6M7pJLABmfBRoGZMu76Y,"[34359739337, 34359738397, 17179869700, 343597...",2310.0,NaN,NaN,NaN,NaN,69GGBxA162lTqCwzJG5jLp,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Gold Digger,5XJJdNPkwmbUwE79gv0NxK,"[17179869726, 34359739337, 34359738917, 190, 3...",2229.0,NaN,NaN,NaN,NaN,5K4W6rqBFWDnAN6FQUkS6x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Redbone,3kxfsdsCpFgN412fpnW85Y,"[17179869726, 34359739337, 34359738917, 190, 1...",2233.0,NaN,NaN,NaN,NaN,73sIBHcqh3Z3NyqHKZ7FOL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Despacito - Remix,5CtI0qwDJkDQGwXD1H1cLb,"[34359739337, 190, 34359738397, 34359738448, 3...",2233.0,NaN,NaN,NaN,NaN,4V8Sr092TqfHkfAA5fXXqG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681800,Intro Bonito,19pUiFzM1Lfui5IzvbJInJ,[8589935151],1.0,NaN,NaN,NaN,NaN,6OqhFYFJDnBBHas02HopPT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681801,Intro / Mule Skinner Blues (Blue Yodel No. 8) ...,3IOJXkHcwFDTOSLxcAw2XF,[25769804664],1.0,NaN,NaN,NaN,NaN,64vAECmFoB6mi7n1zTRwR8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681802,Intro / Madness,62KdOlGDRX7jbQJLIivG5j,[8589934677],1.0,NaN,NaN,NaN,NaN,75S63f1AmZUa9gpQvlt5NB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
681803,Intro - Urban Salute to Hector Lavoe,5krfvR6iPmNbXalFgmtvPS,[17179869244],1.0,NaN,NaN,NaN,NaN,6P9Adm5Ne2YtzhV1hOjQcC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
x


(61415, 22)

In [125]:
# Save the dataframe to a csv file
df_final_metadata.to_csv('Dataset/1,2M_Tracks_with_playlists.csv', index=False)